# User Guide

:::{toctree}
:titlesonly:
:maxdepth: 2
:hidden:

api
:::

`xarray-jsonschema` provides a set of model classes for describing [xarray](https://xarray.dev/) objects and their key features. You can use these models to validate your xarray objects. 

Let's start with an example {py:class}`DataArray`.

In [1]:
import numpy as np
import xarray as xr

da = xr.DataArray(
    np.ones((4, 10), dtype='i4'),
    dims=('x', 't'),
    name='foo',
    attrs={'spam': True, 'eggs': 42},
)

We can use the {py:class}`~.DataArrayModel` class to create a model for this `DataArray`. The model's arguments correspond to the features you want to validate.

In [2]:
from xarray_jsonschema import DataArrayModel

model = DataArrayModel(
    dtype=np.int32,
    dims=['x', 't'],
    name='foo',
    shape=(4, ...),  #  The ellipsis (``...``) is used as a wildcard
)

Models expose a `a {py:meth}`~.Model.validate` method that accepts the object you want to to validate.

In [3]:
model.validate(da)

You can also call the model directly, which is equivalent to calling `validate`:

In [4]:
model(da)

If validation is successful, nothing happens. Otherwise, a {py:class}`~.ValidationError` is raised:

In [5]:
from xarray_jsonschema import ValidationError

da2 = da.astype('int64')

try:
    model.validate(da2)
except ValidationError as error:
    print(error.message)

'int32' was expected


## Composing Schemas

In addition to models like {py:class}`~.DataArrayModel` and {py:class}`~xarray.DatasetModel`, `xarray_jsonschema` also provides models for the individual features of these data structures.

Most of the key features of the `xarray` [data model](https://docs.xarray.dev/en/stable/user-guide/index.html) are implemented as standalone classes.

In [13]:
from xarray_jsonschema import (
    CoordsModel,
    DataVarsModel,
    AttrsModel,
    DimsModel,
    DTypeModel,
    NameModel,
    ShapeModel,
)

name = NameModel.regex('^fo+$')  # some models have special constructors
dims = DimsModel((..., 't'))  # ``...`` is used as a wildcard
dtype = DTypeModel('int32')
shape = ShapeModel((4, 10))
attrs = AttrsModel(
    attrs={'spam': True, 'eggs': int},
)

These models make it easy to create reusable schema that you can compose together.

In [14]:
model = DataArrayModel(
    name=name,
    dims=dims,
    shape=shape,
    dtype=dtype,
    attrs=attrs,
)
model.validate(da)

But they are all fully-fledged models. You can use them to validate object features:

In [ ]:
dims.validate(da.dims)

## JSON Schema

To view the generated JSON Schema, you can call the {py:meth}`~.Model.to_schema` method to return a Python dictionary: 

In [15]:
attrs.to_schema()

{'$schema': 'https://json-schema.org/draft/2020-12/schema',
 'type': 'object',
 'properties': {'spam': {'const': True}, 'eggs': {'type': 'integer'}},
 'required': ['eggs', 'spam']}

Or the {py:meth}`~.Model.to_json` method to serialize the schema to a JSON formatted string:

In [17]:
print(dims.to_json(indent=2))

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "array",
  "prefixItems": [
    {},
    {
      "const": "t"
    }
  ],
  "minItems": 2,
  "maxItems": 2
}
